### Jul AdventKalender D19

https://adventofcode.com/2023/day/19

#### Part 1 

In [3]:
import re

rule_pattern = r'(\w+)\s*([<>]=?|==)\s*([0-9]+):([^\s,]+)'
part_pattern = r'(\w+)\s*=\s*([0-9]+)'
workflows = {}
parts = []

with open("data/input19.txt","r") as file:
    is_workflow=True
    for line in file.readlines():
        line = line.strip()
        if len(line)==0:
            is_workflow=False
            continue
        if is_workflow:
            key, values = line.split('{')
            workflows[key] = re.findall(rule_pattern, values)
            workflows[key].append(values[:-1].split(',')[-1]) # last rule
        else:
            sub_parts = re.findall(part_pattern, line)
            part = {}
            for sub_part in sub_parts:
                part[sub_part[0]]=sub_part[1] 
            parts.append(part)

In [4]:
import operator
op_dict = {
    '>': operator.gt,
    '<': operator.lt
}

def workflow_rule_checker(part, workflow_name):
    rules = workflows[workflow_name]
    for rule in rules[:-1]: #('a', '<', '2006', 'qkq')
        if op_dict.get(rule[1])(int(part[rule[0]]), int(rule[2])):
            return rule[3]
    return rules[-1]

def workflow_end(part, workflow_name):
    if workflow_name in ['A','R']:
        res=0
        if workflow_name == 'A': # accept
            res = sum(map(int,part.values()))
        return True, res
    return False, 0

def validate_part(part):
    next_workflow = 'in'
    while (True):
        is_end, res = workflow_end(part, next_workflow)
        if is_end:
            break
        next_workflow = workflow_rule_checker(part, next_workflow)
    return res

In [5]:
sum_total = 0
for part in parts:
    sum_total+=validate_part(part)
sum_total

389114

#### Part 2

In [22]:
from functools import reduce
from operator import mul

def workflow_validate(workflow_name, candidates):
    if workflow_name == 'A':
        return reduce(mul, [len(sub_part_values) for sub_part_name, sub_part_values in candidates.items()])
    if workflow_name == 'R':
        return 0
    res = 0
    workflow_rules = workflows[workflow_name]
    for rule in workflow_rules[:-1]: #('a', '<', '2006', 'qkq')
        candidates_fulfilled = candidates.copy()
        filter_fun = (lambda x: x < int(rule[2])) if rule[1] == '<' else (lambda x: x > int(rule[2]))
        
        candidates_fulfilled[rule[0]] = list(filter(filter_fun, candidates_fulfilled[rule[0]])) # fulfilled candidates
        candidates[rule[0]] = list(filter(lambda x: not filter_fun(x), candidates[rule[0]])) # not fulfilled, go to next rule
        res += workflow_validate(rule[3], candidates_fulfilled)
    return res + workflow_validate(workflow_rules[-1], candidates) # last rule

candidates_original = {'x':[i for i in range(1, 4001)],
                       'm':[i for i in range(1, 4001)],
                       'a':[i for i in range(1, 4001)],
                       's':[i for i in range(1, 4001)]}
workflow_validate('in', candidates_original)

125051049836302